In [1]:
import read_data_utils as utils
import regression_weight_tuning_EU_LG_UA as EU_LG_UA
import reorganization as reorg

In [2]:
import sys
import copy
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [3]:
file = 'train_all_0.csv'
dataloaders, dataset_sizes = utils.read_data(file, batch_size=4, mode='mse')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = nn.Sequential(
      nn.Linear(12,128),
      nn.ReLU(),
      nn.Linear(128,1)).to(device)

# model.load_state_dict(torch.load('../acceptable_weights/before_reorg.pt'))

criterion = nn.MSELoss()

train_result = EU_LG_UA.train_model(model, criterion, dataloaders, dataset_sizes, device,
                                    PATH = '../weights/train_checkpoint.pt', epsilon = 1e-6, num_epochs=50, n=5)

if train_result['result']:
    print('Acceptable SLFN')
    torch.save(model.state_dict(), '../acceptable_weights/before_reorg.pt')
#     model.load_state_dict(torch.load('../acceptable_weights/before_reorg.pt'))
    reorg_result, p = reorg.reorg_model(model, criterion, dataloaders, dataset_sizes, device, 
                                      PATH='../weights/reorg_checkpoint.pt', lr_epsilon=1e-6 , n=5, rs=0.001)
    print(reorg_result, p)
    torch.save(result, f'../acceptable_weights/after_reorganized_{p}.pt')
else:
    print('Unacceptable SLFN')